In [1]:
# 모듈 import
from mmcv import Config
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor
from mmseg.datasets import build_dataloader, build_dataset   

In [2]:
classes = ("Backgroud", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
palette = [[0,0,0], [192,0,128], [0,128,192], [0,128,64], [128,0,0], [64,0,128],
           [64,0,192] ,[192,128,64], [192,192,128], [64,64,128], [128,0,192]]

# config file 들고오기
cfg_file = 'upernet_swin_large_patch4_window12_512x512_80k_coco_pretrain_384x384_22K'
cfg = Config.fromfile(f'./configs/visual_trash_seg/models/{cfg_file}.py')

# root = '../input/data/'

# dataset config 수정
# cfg.data.train.classes = classes
# cfg.data.train.data_root = root
cfg.data.train.img_dir = 'pseudo/img'
cfg.data.train.ann_dir = 'pseudo/ann'
# cfg.data.train.ann_file = root + 'train.json' # train json 정보
# cfg.data.train.pipeline[2]['img_scale'] = (512,512)

# cfg.data.test.classes = classes
# cfg.data.test.data_root = root
# cfg.data.test.img_dir = 'test/img/'
# cfg.data.test.ann_file = root + 'test.json' # test json 정보
# cfg.data.test.pipeline[1]['img_scale'] = (512,512)

cfg.data.samples_per_gpu = 4

cfg.seed = 1995
cfg.gpu_ids = [0]
cfg.work_dir = f'./work_dirs/{cfg_file}_pseudo'

cfg.log_config.hooks[1].init_kwargs.name = 'swin_l_pseudo_384'
# cfg.model.roi_head.bbox_head.num_classes = 10
# cfg.model.bbox_head.num_classes = 10

# cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
# cfg.checkpoint_config = dict(max_keep_ckpts=10, interval=1)

In [3]:
cfg.data.train

{'type': 'COCOTrashDataset',
 'data_root': '../input/data',
 'reduce_zero_label': False,
 'img_dir': 'pseudo/img',
 'ann_dir': 'pseudo/ann',
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'LoadAnnotations'},
  {'type': 'Resize', 'img_scale': (2048, 512), 'ratio_range': (0.5, 2.0)},
  {'type': 'RandomCrop', 'crop_size': (384, 384), 'cat_max_ratio': 0.75},
  {'type': 'RandomFlip', 'prob': 0.5},
  {'type': 'PhotoMetricDistortion'},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_rgb': True},
  {'type': 'Pad', 'size': (384, 384), 'pad_val': 0, 'seg_pad_val': 255},
  {'type': 'DefaultFormatBundle'},
  {'type': 'Collect', 'keys': ['img', 'gt_semantic_seg']}]}

In [4]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

2021-11-02 06:02:54,827 - mmseg - INFO - Loaded 4091 images


In [8]:
# dataset 확인
datasets[0]

In [5]:
# 모델 build 및 pretrained network 불러오기
model = build_segmentor(cfg.model)
model.init_weights()

/opt/ml/segmentation/semantic-segmentation-level2-cv-17/mmsegmentation/mmseg/models/backbones/swin.py:555: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
2021-11-02 06:03:15,794 - mmseg - INFO - Use load_from_http loader
2021-11-02 06:03:16,541 - mmseg - INFO - initialize UPerHead with init_cfg {'type': 'Normal', 'std': 0.01, 'override': {'name': 'conv_seg'}}
2021-11-02 06:03:16,848 - mmseg - INFO - initialize FCNHead with init_cfg {'type': 'Normal', 'std': 0.01, 'override': {'name': 'conv_seg'}}
2021-11-02 06:03:16,852 - mmseg - INFO - 
backbone.patch_embed.projection.weight - torch.Size([192, 3, 4, 4]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
2021-11-02 06:03:16,853 - mmseg - INFO - 
backbone.patch_embed.projection.bias - torch.Size([192]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
2021-11-02 06:03:16,853 - mmseg - INFO - 
back

In [6]:
# 모델 학습

train_segmentor(model, datasets[0], cfg, distributed=False, validate=False)

2021-11-02 06:03:26,411 - mmseg - INFO - Start running, host: root@9760b71bd07f, work_dir: /opt/ml/segmentation/semantic-segmentation-level2-cv-17/mmsegmentation/work_dirs/upernet_swin_large_patch4_window12_512x512_80k_coco_pretrain_384x384_22K_pseudo
2021-11-02 06:03:26,413 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMA

2021-11-02 06:04:22,503 - mmseg - INFO - Iter [50/80000]	lr: 9.995e-03, eta: 22:12:39, time: 1.000, data_time: 0.010, memory: 17737, decode.loss_ce: 1.2251, decode.acc_seg: 57.5293, aux.loss_ce: 0.5489, aux.acc_seg: 52.8081, loss: 1.7739
2021-11-02 06:05:12,238 - mmseg - INFO - Iter [100/80000]	lr: 9.989e-03, eta: 22:08:13, time: 0.995, data_time: 0.006, memory: 17737, decode.loss_ce: 0.9118, decode.acc_seg: 67.2784, aux.loss_ce: 0.3888, aux.acc_seg: 66.1264, loss: 1.3005
2021-11-02 06:06:02,043 - mmseg - INFO - Iter [150/80000]	lr: 9.983e-03, eta: 22:06:48, time: 0.996, data_time: 0.006, memory: 17737, decode.loss_ce: 0.7790, decode.acc_seg: 71.4985, aux.loss_ce: 0.3284, aux.acc_seg: 71.5497, loss: 1.1074
2021-11-02 06:06:51,981 - mmseg - INFO - Iter [200/80000]	lr: 9.978e-03, eta: 22:06:33, time: 0.999, data_time: 0.006, memory: 17737, decode.loss_ce: 0.7789, decode.acc_seg: 71.5680, aux.loss_ce: 0.3381, aux.acc_seg: 69.8245, loss: 1.1169
2021-11-02 06:07:41,846 - mmseg - INFO - Iter

KeyboardInterrupt: 

In [ ]:
!python ./tools/analysis_tools/analyze_logs.py plot_curve ./work_dirs/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco/None.log.json --keys loss --out losses.png

plot curve of ./work_dirs/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco/None.log.json, metric is loss
save curve to: losses.png


In [8]:
!python ./tools/analysis_tools/analyze_logs.py plot_curve ./work_dirs/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco/None.log.json --keys loss_cls loss_bbox --out losses.png

plot curve of ./work_dirs/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco/None.log.json, metric is loss_cls
Traceback (most recent call last):
  File "./tools/analysis_tools/analyze_logs.py", line 180, in <module>
    main()
  File "./tools/analysis_tools/analyze_logs.py", line 176, in main
    eval(args.task)(log_dicts, args)
  File "./tools/analysis_tools/analyze_logs.py", line 55, in plot_curve
    f'{args.json_logs[i]} does not contain metric {metric}')
KeyError: './work_dirs/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco/None.log.json does not contain metric loss_cls'
